# Pré-processamento dos dados do Vivino para input no Tableau

Este notebook descreve o passo-a-passo da preparação dos dados que baixei do meu perfil no Vivino em Junho de 2020, com o objetivo de construir um dashboard no Tableau para fins de análise exploratória.

Este trabalho foi realizado em R. Para a instalação do kernel do R no Jupyter, o seguintes tutoriais podem ser úteis:

- https://medium.com/@kyleake/how-to-install-r-in-jupyter-with-irkernel-in-3-steps-917519326e41
- https://richpauloo.github.io/2018-05-16-Installing-the-R-kernel-in-Jupyter-Lab/

O Vivino é um app onde os usuários podem catalogar, dar uma nota e escrever um review para os vinhos que degusta a partir do upload de uma foto do rótulo. Mais informações podem ser obtidas na Wikipedia: https://en.wikipedia.org/wiki/Vivino

Para realizar esta tarefa, optei por utilizar a biblioteca dplyr do R:

In [1]:
library(dplyr)

Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Em seguida carreguei o arquivo de interesse (full_wine_list.csv) que baixei do Vivino e verifiquei a sua estrutura e 
características gerais:

In [2]:
dados <- read.csv(file.choose())

head(dados,1)
str(dados)
summary(dados)




Winery,Wine.name,Vintage,Region,Country,Regional.wine.style,Average.rating,Scan.date,Scan.Review.Location,Your.rating,Your.review,Personal.Note,Wine.type,Drinking.Window,Wishlisted.date,Cellar.count,Link.to.wine,Label.image,Wine.price
Matetic,EQ Granite Pinot Noir,2014,Casablanca Valley,Chile,Chilean Pinot Noir,3.97,2020-06-16 14:36:21,NA,4,,NA,Red Wine,NA,NA,NA,https://www.vivino.com/wines/160547630,https://images.vivino.com/labels/FGBh-GGJQJG3hZNbVSZljw.jpg,


'data.frame':	87 obs. of  19 variables:
 $ Winery              : Factor w/ 78 levels "Alamos","Albert Bichot",..: 50 3 61 38 8 73 9 23 25 75 ...
 $ Wine.name           : Factor w/ 75 levels "Alta Malbec",..: 27 32 67 24 53 9 68 74 8 65 ...
 $ Vintage             : int  2014 2018 2016 2017 2014 8888 2012 2018 2008 2017 ...
 $ Region              : Factor w/ 49 levels "","Aconcagua Valley",..: 14 41 14 46 11 36 30 3 48 25 ...
 $ Country             : Factor w/ 13 levels "","Argentina",..: 4 2 4 2 12 3 12 9 5 4 ...
 $ Regional.wine.style : Factor w/ 36 levels "","Argentinian Cabernet Sauvignon",..: 22 4 22 2 14 10 15 29 1 20 ...
 $ Average.rating      : num  3.97 3.41 3.62 3.39 3.54 3.27 3.82 3.63 3.46 3.54 ...
 $ Scan.date           : Factor w/ 87 levels "","2014-11-09 22:25:45",..: 87 86 85 84 83 82 81 80 79 78 ...
 $ Scan.Review.Location: logi  NA NA NA NA NA NA ...
 $ Your.rating         : num  4 3 NA 3 3.5 3 4 3.5 3 3 ...
 $ Your.review         : Factor w/ 8 levels "","!!!","Ã\200 su

                  Winery                             Wine.name     Vintage    
 TarapacÃ¡           : 3   Cabernet Sauvignon             : 5   Min.   :2008  
 Alfredo Roca        : 2   Pinot Noir                     : 4   1st Qu.:2012  
 Casillero del Diablo: 2   Gran Reserva Cabernet Sauvignon: 3   Median :2014  
 CousiÃ±o-Macul      : 2   Fincas Malbec                  : 2   Mean   :2263  
 D.V. Catena         : 2   Malbec                         : 2   3rd Qu.:2016  
 Luigi Bosca         : 2   Reserva Malbec                 : 2   Max.   :9999  
 (Other)             :74   (Other)                        :69                 
               Region            Country  
 Mendoza          :13   Chile        :28  
 Colchagua Valley : 8   Argentina    :26  
 Maipo Valley     : 5   France       : 9  
 Lujan de Cuyo    : 4   Italy        : 5  
 Casablanca Valley: 3   Spain        : 4  
 Central Valley   : 3   United States: 4  
 (Other)          :51   (Other)      :11  
                     Reg

O resultado mostra que várias colunas do arquivo baixado não terão utilidade na minha análise. 

Por exemplo, colunas como "Drinking.Window","Scan.Review.Location", dentre outras não possuem valor em nenhum dos registros, pois são campos que não preenchi quando fiz a avaliação dos rótulos.

No trecho de código abaixo retirei todas as colunas que não são relevantes:

In [3]:
dados <- dados %>% 
  select(-c(Scan.Review.Location,Your.review,Personal.Note,Drinking.Window,
            Wishlisted.date,Cellar.count,Wine.price))

O próximo passo é verificar a qualidade dos dados das colunas remanescentes, bem como corrigir possíveis inconsistências.

Vamos começar com a coluna "Country": o comando summary nos informou a existência de 17 países distintos e vamos verificar quais são eles:

In [4]:
dados %>% distinct(Country) %>% arrange(Country)

Country
""
Argentina
Brazil
Chile
France
Germany
Italy
New Zealand
Portugal
South Africa


Como um dos valores únicos de país está em branco, vamos verificar quantas entradas são afetadas:

In [5]:
dados %>% filter(Country=="")

Winery,Wine.name,Vintage,Region,Country,Regional.wine.style,Average.rating,Scan.date,Your.rating,Wine.type,Link.to.wine,Label.image
Steakhouse,Brett Favre's Cabernet Sauvignon,2016,,,,3.13,2017-09-30 22:35:10,NA,Red Wine,https://www.vivino.com/wines/154556162,https://images.vivino.com/labels/hmTFCi4SSDiVv5RJ6w-odQ.jpg


Apenas 1 das entradas foi afetada e, por inspeção manual deste vinho no Vivino, verifiquei que trata-se de um rótulo produzido na Argentina. Desta forma, vamos corrigir o registro:

In [6]:
dados <- dados %>%
  mutate(Country=replace(Country,which(Country==""),"Argentina"))

Em seguida vamos verificar o campo "Vintage", pois o comando Summary indicou que o valor máximo deste campo é 9999, o que não é condizente com o ano de safra de um vinho. Vamos verificar os valores únicos:

In [7]:
dados %>% distinct(Vintage) %>% arrange(Vintage)

Vintage
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017


Em seguida vamos buscar as entradas afetadas pelos valores 8888 e 9999:

In [8]:
dados %>% filter(Vintage == 8888 | Vintage == 9999)

Winery,Wine.name,Vintage,Region,Country,Regional.wine.style,Average.rating,Scan.date,Your.rating,Wine.type,Link.to.wine,Label.image
VinÃ­cola Hermann,Bossa No. 1 Brut,8888,Rio Grande do Sul,Brazil,Brazilian Sparkling,3.27,2020-01-10 00:34:27,3,Sparkling,https://www.vivino.com/wines/2183905,https://images.vivino.com/labels/iTw1X0jNSne-pOcf8Hmabg.jpg
I Castelli di Romeo e Giulietta,Bardolino,9999,Bardolino,Italy,Northern Italy Red,3.23,2019-03-28 18:53:08,3,Red Wine,https://www.vivino.com/wines/4433694,https://images.vivino.com/labels/Kxjmg2jfQbq5w5XqNlcr3g.jpg
Col Brioso,Gran CuvÃ©e Extra Dry,8888,Prosecco,Italy,Italian Prosecco,3.58,2018-12-21 23:03:12,3,Sparkling,https://www.vivino.com/wines/152392933,https://images.vivino.com/labels/jFay58HSQ6OhPJpUs9mxrw.jpg


Concluí que o valor 8888 está atrelado a espumantes e, para esta categoria,especificar a safra não é aplicável. Já o valor 9999 refere-se a um vinho tinto e, ao verificar manualmente no Vivino, não consegui encontrar a safra no rótulo. Deixei estes valores para serem tratados posteriormente no Tableau (utilizarei um Alias para substituí-los nos gráficos).

Os próximos campos a serem analisados são "Average.rating" e "Your.rating", pois uma das coisas que quero saber é se minhas avaliações são melhores ou piores que a média de avaliações do Vivino.

A saída do comando Summary mostrou que o campo "Average.rating" está ok, pois todos os valores estão presentes e contidos no intervalo de 0 a 5, nota mínima e máxima que pode ser dada a um rótulo, respectivamente. Já o "Your.rating" possui NAs e vamos verificar os registros afetados:

In [9]:
dados %>% filter(is.na(Your.rating))

Winery,Wine.name,Vintage,Region,Country,Regional.wine.style,Average.rating,Scan.date,Your.rating,Wine.type,Link.to.wine,Label.image
Sideways,Signature Series Pinot Noir,2016,Casablanca Valley,Chile,Chilean Pinot Noir,3.62,2020-01-26 17:39:44,NA,Red Wine,https://www.vivino.com/wines/161911521,https://images.vivino.com/labels/tN8KLpyRSoiMmYJ9F18Znw.jpg
Los Riscos,Cabernet Sauvignon,2016,Curico Valley,Chile,Chilean Cabernet Sauvignon,3.18,2018-01-27 20:46:44,NA,Red Wine,https://www.vivino.com/wines/99118949,https://images.vivino.com/labels/2wH7dxWsQJW7hhLu1mJk7Q.jpg
Steakhouse,Brett Favre's Cabernet Sauvignon,2016,,Argentina,,3.13,2017-09-30 22:35:10,NA,Red Wine,https://www.vivino.com/wines/154556162,https://images.vivino.com/labels/hmTFCi4SSDiVv5RJ6w-odQ.jpg
TarapacÃ¡,Terroir El Rosal Pinot Noir,2013,Casablanca Valley,Chile,Chilean Pinot Noir,3.57,2016-03-05 19:17:50,NA,Red Wine,https://www.vivino.com/wines/7300440,https://images.vivino.com/labels/VD9ouAVNRTybvSP01AO9KQ.jpg


Verifiquei no Vivino que, de fato, minha avaliação não ficou registrada para estes rótulos. Visto que normalmente dou nota 3 nos vinhos que acho bons para o dia a dia e que, seu eu achasse algo especial nesses rótulos a ponto de elevar a nota, eu dificilmente teria deixado de registrar resolvi aplicar a nota 3 a todos eles.

In [10]:
table(dados$Your.rating)

dados <- dados %>%
  mutate(Your.rating=replace(Your.rating,which(is.na(Your.rating)),3))


1.5   3 3.5   4 4.5   5 
  1  41  18  17   5   1 

Por último analisei a qualidade do campo "Scan.date", pois pretendo filtrar as análises por ano de degustação:

In [11]:
dados %>% distinct(Scan.date)%>% arrange(Scan.date)

Scan.date
""
2014-11-09 22:25:45
2014-12-24 15:02:20
2014-12-25 16:08:10
2014-12-31 23:44:12
2015-02-16 01:13:01
2015-02-16 15:00:13
2015-03-01 23:30:27
2015-04-18 16:34:29
2015-05-03 22:38:55


Visto que temos uma data em branco vamos verificar qual é o registro afetado:

In [12]:
dados %>% filter(Scan.date=="")

Winery,Wine.name,Vintage,Region,Country,Regional.wine.style,Average.rating,Scan.date,Your.rating,Wine.type,Link.to.wine,Label.image
Goldridge Estate,Pinot Noir,2011,Marlborough,New Zealand,New Zealand Pinot Noir,3.3,,1.5,Red Wine,https://www.vivino.com/wines/1741900,


Verifiquei que a data está preenchida no Vivino, apesar de não estar nos dados que baixei: 28 de agosto de 2015. Em seguida realizei a correção do registro:

In [13]:
dados$Scan.date <- as.character(dados$Scan.date)

dados <- dados %>%
  mutate(Scan.date=replace(Scan.date,which(Scan.date==""),"2015-08-28 00:00:00"))


Por fim vamos salvar os dados em um arquivo csv para dar início ao trabalho no Tableau, já que as variáveis que potencialmente tem mais impacto foram saneadas:

In [ ]:
write.csv(dados,"vinhos_vivino_jun_20.csv")

Caso você queira ver como ficou o dashboard no Tableau, basta accessá-lo no link:
- https://public.tableau.com/profile/debora.coutinho#!/vizhome/Vivino_dashboard/VivinoDashboard?publish=yes